In [2]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
import matplotlib.pyplot as plt
import scipy.stats as sct
from numba import jit, njit, prange




In [3]:

log_e = np.round(np.arange(2, 10.2, 0.2), 2) #log10(E/GeV) values range as in all 'effectiveArea' files
e_nu_wall = np.asarray((10**log_e) * 1e9)
dec_nu = [-90.  , -73.74, -66.93, -61.64, -57.14, -53.13, -49.46, -46.05,
       -42.84, -39.79, -36.87, -34.06, -31.33, -28.69, -26.1 , -23.58,
       -21.1 , -18.66, -16.26, -13.89, -11.54,  -9.21,  -6.89,  -4.59,
        -2.29,   0.  ,   2.29,   4.59,   6.89,   9.21,  11.54,  13.89,
        16.26,  18.66,  21.1 ,  23.58,  26.1 ,  28.69,  31.33,  34.06,
        36.87,  39.79,  42.84,  46.05,  49.46,  53.13,  57.14,  61.64,
        66.93,  73.74,  90.  ]


dec_nu = np.asfarray(dec_nu)
# UNCOMMENT FOR LINEAR BINS
# all_enu = np.linspace(10**11.001, 10**18.999, 1000)

# UNCOMMENT FOR LOGARITHMIC BINS of WIDTH 0.2 from logE/GeV = 2 to 10 AS GIVEN IN THE ICECUBE DATA
all_enu = e_nu_wall

# UNCOMMENT FOR LOGARITHMIC BINS
# all_enu = np.logspace(11.001, 18.999, 1000)
gamma_arr = [-1, -2.2, -2.5, -3]
phio = np.logspace(-38, -26, 1000) #CHANGING TO LINEAR BINS RESULTS IN STRAIGHT LINES

t_upt = 35318439.34329611

In [4]:
earea = "0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.132710000000000036e+03,2.236670000000000073e+03,1.250897999999999956e+04,3.538876000000000204e+04,8.675422000000000116e+04,1.984782999999999884e+05,3.760966799999999930e+05,5.449500400000000373e+05,8.308070799999999581e+05,9.934056999999999534e+05,1.225245800000000047e+06,1.415589260000000009e+06,1.633582750000000000e+06,1.671608419999999925e+06,1.478340889999999898e+06,1.226410060000000056e+06,1.002361479999999981e+06,7.709989499999999534e+05,6.686725100000000093e+05,4.459323200000000070e+05,4.228735700000000070e+05,3.886283099999999977e+05,2.870765599999999977e+05,1.688355599999999977e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.267300000000000040e+02,1.484980000000000018e+03,5.634010000000000218e+03,1.795386999999999898e+04,4.665161000000000058e+04,1.113645000000000000e+05,2.742375200000000186e+05,4.351150000000000000e+05,6.324551300000000047e+05,9.242513199999999488e+05,1.098512419999999925e+06,1.257085979999999981e+06,1.506695479999999981e+06,1.599262760000000009e+06,1.833570219999999972e+06,1.669699590000000084e+06,1.223314800000000047e+06,9.428664799999999814e+05,7.004241800000000512e+05,3.555155999999999767e+05,4.109181300000000047e+05,4.024509799999999814e+05,2.634330100000000093e+05,1.791803399999999965e+05,2.346616900000000023e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,3.443899999999999864e+02,1.372539999999999964e+03,9.140870000000000800e+03,1.855443999999999869e+04,5.250347000000000116e+04,1.321390799999999872e+05,3.010928800000000047e+05,5.085119000000000233e+05,7.286286800000000512e+05,9.962000799999999581e+05,1.315495540000000037e+06,1.458978379999999888e+06,1.603563379999999888e+06,1.843221800000000047e+06,1.772051030000000028e+06,1.683086360000000102e+06,1.337173629999999888e+06,1.212705340000000084e+06,7.932045000000000000e+05,5.765853299999999581e+05,4.224740999999999767e+05,3.159711500000000233e+05,2.649334500000000116e+05,1.597165499999999884e+05,6.191568000000000029e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.391200000000000045e+02,2.749610000000000127e+03,7.089170000000000073e+03,2.029099000000000160e+04,6.960552999999999884e+04,1.579440000000000000e+05,3.227177199999999721e+05,5.171252600000000093e+05,8.125545400000000373e+05,1.067242709999999963e+06,1.280698790000000037e+06,1.537518629999999888e+06,1.920013580000000075e+06,1.890267929999999935e+06,2.090880100000000093e+06,1.739203129999999888e+06,1.705660760000000009e+06,1.159734679999999935e+06,9.984090899999999674e+05,5.729595600000000559e+05,5.095914899999999907e+05,5.608209100000000326e+05,2.524298599999999860e+05,2.810027399999999907e+05,1.782587600000000093e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.154399999999999977e+02,0.000000000000000000e+00,3.883700000000000045e+02,2.959230000000000018e+03,7.817930000000000291e+03,2.753550999999999840e+04,7.114322999999999593e+04,1.586562500000000000e+05,3.553669299999999930e+05,5.812356199999999953e+05,8.013253399999999674e+05,1.072996370000000112e+06,1.424983550000000047e+06,1.769098149999999907e+06,1.986309080000000075e+06,2.097182069999999832e+06,2.163172720000000205e+06,2.113776919999999925e+06,1.691787050000000047e+06,1.344903929999999935e+06,1.079065159999999916e+06,8.385628299999999581e+05,5.837835600000000559e+05,3.863550399999999790e+05,4.971887999999999884e+05,3.803419400000000023e+05,3.090632899999999790e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.514199999999999591e+02,2.846730000000000018e+03,1.511210000000000036e+04,3.322552999999999884e+04,6.983216999999999825e+04,1.888032799999999988e+05,3.995102199999999721e+05,6.643660799999999581e+05,9.335601700000000419e+05,1.287165219999999972e+06,1.556408540000000037e+06,1.880165570000000065e+06,2.028342870000000112e+06,2.349451700000000186e+06,2.283801959999999963e+06,2.377357750000000000e+06,2.086330500000000000e+06,1.496467199999999953e+06,1.241886899999999907e+06,8.619696800000000512e+05,6.087310400000000373e+05,4.842843499999999767e+05,4.416304899999999907e+05,4.112302500000000000e+05,2.092849299999999930e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,7.106599999999999682e+02,3.841440000000000055e+03,1.111352000000000044e+04,3.042452999999999884e+04,9.106246000000000640e+04,1.955875599999999977e+05,4.330742100000000210e+05,7.112027800000000279e+05,1.051415780000000028e+06,1.326822409999999916e+06,1.669103399999999907e+06,2.034730030000000028e+06,2.461418669999999925e+06,2.620172060000000056e+06,2.654520100000000093e+06,2.754822819999999832e+06,2.336882950000000186e+06,1.908630469999999972e+06,1.348903479999999981e+06,1.037087560000000056e+06,7.948265400000000373e+05,6.665693399999999674e+05,5.311589599999999627e+05,4.652718200000000070e+05,2.752843800000000047e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.553499999999999943e+02,9.824600000000000364e+02,4.302949999999999818e+03,1.655388999999999942e+04,3.825106999999999971e+04,9.321158000000000175e+04,2.367620000000000000e+05,4.876888499999999767e+05,8.348042299999999814e+05,1.199631540000000037e+06,1.493274379999999888e+06,1.839797399999999907e+06,2.302805229999999981e+06,2.470512750000000000e+06,2.761640200000000186e+06,2.972616879999999888e+06,3.132736129999999888e+06,2.604105049999999814e+06,2.039048489999999991e+06,1.865777889999999898e+06,1.374114080000000075e+06,8.884282600000000093e+05,9.120912299999999814e+05,6.413094799999999814e+05,5.001805300000000279e+05,4.037295999999999767e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.373899999999999864e+02,9.785499999999999545e+02,3.292010000000000218e+03,1.217382999999999993e+04,3.543480999999999767e+04,1.065862500000000000e+05,2.358699400000000023e+05,5.190635000000000000e+05,9.052609799999999814e+05,1.182313600000000093e+06,1.602238100000000093e+06,1.962546629999999888e+06,2.414712540000000037e+06,2.884685009999999776e+06,3.124211189999999944e+06,3.563474839999999851e+06,3.288321540000000037e+06,2.877077459999999963e+06,2.489674459999999963e+06,1.993740129999999888e+06,1.792207770000000019e+06,1.458282570000000065e+06,1.003670640000000014e+06,7.845071300000000047e+05,7.343982399999999907e+05,5.345856500000000233e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.222900000000000063e+02,1.462589999999999918e+03,4.511640000000000327e+03,1.478123999999999978e+04,4.880113999999999942e+04,1.281235599999999977e+05,2.715416099999999860e+05,5.827350799999999581e+05,9.799635799999999581e+05,1.247298639999999898e+06,1.734865820000000065e+06,2.121359049999999814e+06,2.632286109999999870e+06,3.153424459999999963e+06,3.244413149999999907e+06,3.898445120000000112e+06,3.771488120000000112e+06,3.385091479999999981e+06,3.003605720000000205e+06,2.419985970000000205e+06,1.924495290000000037e+06,1.665923489999999991e+06,1.002095900000000023e+06,1.005629630000000005e+06,7.595482900000000373e+05,6.176260100000000093e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.360399999999999636e+02,2.477769999999999982e+03,7.034739999999999782e+03,1.566465999999999985e+04,5.725926000000000204e+04,1.394859899999999907e+05,3.225741699999999837e+05,6.518875500000000466e+05,1.047113030000000028e+06,1.491406610000000102e+06,1.923008939999999944e+06,2.437474240000000224e+06,2.895804390000000130e+06,3.253525569999999832e+06,3.889176700000000186e+06,4.340515860000000335e+06,4.221626540000000037e+06,3.743705629999999888e+06,3.693844140000000130e+06,3.123959649999999907e+06,2.144578770000000019e+06,1.942774870000000112e+06,1.824960360000000102e+06,1.212377689999999944e+06,1.125084879999999888e+06,7.650593900000000140e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,6.163300000000000409e+02,1.937269999999999982e+03,8.724889999999999418e+03,2.152631999999999971e+04,6.613858000000000175e+04,1.665493299999999872e+05,3.784052299999999814e+05,7.147022199999999721e+05,1.055867360000000102e+06,1.567110449999999953e+06,2.120647640000000130e+06,2.558779000000000000e+06,3.273174359999999870e+06,3.649812029999999795e+06,4.339678179999999702e+06,4.592804559999999590e+06,4.900635570000000298e+06,4.536029019999999553e+06,4.070640339999999851e+06,3.808112310000000056e+06,3.024328490000000224e+06,2.331008569999999832e+06,2.140884970000000205e+06,1.354048129999999888e+06,1.500303040000000037e+06,8.919711500000000233e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.178329999999999927e+03,3.173869999999999891e+03,9.537479999999999563e+03,3.083902000000000044e+04,9.011696000000000640e+04,1.995907000000000116e+05,3.870211300000000047e+05,7.749439699999999721e+05,1.180191090000000084e+06,1.729931870000000112e+06,2.417476250000000000e+06,2.726083910000000149e+06,3.495428189999999944e+06,3.925453540000000037e+06,5.047916519999999553e+06,5.043874740000000224e+06,5.368264919999999925e+06,5.402703509999999776e+06,4.980999259999999776e+06,4.503020160000000149e+06,3.861479279999999795e+06,3.229653319999999832e+06,2.479060410000000149e+06,1.964050100000000093e+06,1.598684600000000093e+06,1.558182689999999944e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,2.856399999999999864e+02,1.368529999999999973e+03,3.386010000000000218e+03,9.116479999999999563e+03,3.462251000000000204e+04,8.603091999999999825e+04,2.288739100000000035e+05,4.810980499999999884e+05,8.730116899999999441e+05,1.376018100000000093e+06,1.935288889999999898e+06,2.420939120000000112e+06,3.083738459999999963e+06,3.741712700000000186e+06,4.767318309999999590e+06,5.456609000000000000e+06,6.080360700000000186e+06,6.323248219999999739e+06,6.681295190000000410e+06,5.712418959999999963e+06,5.614098660000000149e+06,4.436939969999999739e+06,3.537163479999999981e+06,3.323003779999999795e+06,2.603556060000000056e+06,2.221100910000000149e+06,1.842140149999999907e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.398799999999999955e+02,2.112820000000000164e+03,4.924630000000000109e+03,1.220763999999999942e+04,4.230811000000000058e+04,1.075199299999999930e+05,2.535767799999999988e+05,5.514840000000000000e+05,9.599169000000000233e+05,1.539961330000000075e+06,2.154323029999999795e+06,2.912668000000000000e+06,3.607687669999999925e+06,4.289783150000000373e+06,5.313607480000000447e+06,6.238447019999999553e+06,7.161470290000000037e+06,7.371463530000000261e+06,7.446346919999999925e+06,6.999071559999999590e+06,6.378092290000000037e+06,5.984492150000000373e+06,4.680619919999999925e+06,4.169809250000000000e+06,3.944668899999999907e+06,3.061842040000000037e+06,2.504164890000000130e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.125519999999999982e+03,2.077889999999999873e+03,8.184659999999999854e+03,1.960588000000000102e+04,5.905491999999999825e+04,1.389315299999999988e+05,3.121070399999999790e+05,6.114910100000000093e+05,1.131147439999999944e+06,1.673725889999999898e+06,2.445969750000000000e+06,3.236919939999999944e+06,3.919590709999999963e+06,5.005779559999999590e+06,6.045672889999999665e+06,6.943815669999999925e+06,8.294623570000000298e+06,8.353692530000000261e+06,8.588349759999999776e+06,8.662422449999999255e+06,8.334628440000000410e+06,7.744917750000000000e+06,6.266571269999999553e+06,5.323306429999999702e+06,5.600241169999999925e+06,4.590191900000000373e+06,3.762583620000000112e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.459500000000000455e+02,3.018809999999999945e+03,8.514469999999999345e+03,2.626038000000000102e+04,7.527397000000000116e+04,1.767186799999999930e+05,4.020557000000000116e+05,7.821118199999999488e+05,1.333377560000000056e+06,1.830329919999999925e+06,2.615042500000000000e+06,3.423917720000000205e+06,4.385396620000000112e+06,5.474769780000000261e+06,6.938246660000000149e+06,7.893487549999999814e+06,8.822886890000000596e+06,9.708709099999999627e+06,1.111991480000000075e+07,1.086246350999999978e+07,1.015311544999999925e+07,9.260315679999999702e+06,8.707608359999999404e+06,7.365739450000000186e+06,8.020938129999999888e+06,6.222424309999999590e+06,6.062896080000000075e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.576390000000000100e+03,4.009000000000000000e+03,1.519805999999999949e+04,3.532495999999999913e+04,9.382539999999999418e+04,2.445869500000000116e+05,5.122900599999999977e+05,8.277056600000000326e+05,1.475925939999999944e+06,2.115684919999999925e+06,2.786712069999999832e+06,4.072283310000000056e+06,4.960938959999999963e+06,6.514363639999999665e+06,7.805485490000000224e+06,9.130158740000000224e+06,1.009948705000000075e+07,1.153414251999999955e+07,1.294949861999999918e+07,1.270094900000000000e+07,1.194741333999999985e+07,1.225110648000000045e+07,1.228007126999999955e+07,1.095954128999999911e+07,1.030689446000000089e+07,9.110666980000000447e+06,8.940176349999999627e+06,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.208400000000000034e+02,1.568589999999999918e+03,6.875720000000000255e+03,1.833502999999999884e+04,4.862269999999999709e+04,1.318008200000000070e+05,3.035993200000000070e+05,5.963220000000000000e+05,9.919188000000000466e+05,1.588859310000000056e+06,2.458702850000000093e+06,3.352448850000000093e+06,4.536051629999999888e+06,5.936564110000000335e+06,7.159898419999999925e+06,9.228130630000000820e+06,1.060248907000000030e+07,1.240510747000000067e+07,1.381433140000000037e+07,1.518184857000000030e+07,1.616653925000000000e+07,1.625837767999999970e+07,1.569267269999999925e+07,1.610139289000000060e+07,1.653644972000000067e+07,1.507007969999999925e+07,1.380895873000000045e+07,1.325362550000000000e+07,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,6.524099999999999682e+02,5.832000000000000455e+02,3.034659999999999854e+03,1.099322999999999956e+04,3.310105000000000291e+04,8.893738999999999942e+04,1.905227900000000081e+05,4.162273200000000070e+05,8.034618900000000140e+05,1.295141149999999907e+06,2.094512179999999935e+06,2.854190720000000205e+06,3.918586350000000093e+06,5.460494419999999925e+06,6.695946410000000149e+06,8.354870919999999925e+06,1.043672550000000000e+07,1.235435678999999911e+07,1.529917885999999940e+07,1.697727776999999955e+07,1.943097251000000164e+07,2.108772623000000045e+07,2.213950117999999970e+07,2.305184446999999881e+07,2.361670844999999925e+07,2.387909164000000060e+07,2.248286342000000179e+07,2.099609510999999940e+07,2.315827946999999881e+07,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,2.413100000000000023e+02,5.700900000000000318e+02,4.836500000000000000e+03,9.086139999999999418e+03,2.570329000000000087e+04,6.913046000000000640e+04,1.799958099999999977e+05,3.208695499999999884e+05,6.150047900000000373e+05,1.046036510000000009e+06,1.624251520000000019e+06,2.469687680000000168e+06,3.512370000000000000e+06,4.815297820000000298e+06,6.416675230000000447e+06,7.992951570000000298e+06,1.029149463000000082e+07,1.273342043999999948e+07,1.514017000000000000e+07,1.864416023999999836e+07,2.086447489000000060e+07,2.508970442000000179e+07,2.878970008999999985e+07,2.937367417999999970e+07,2.889083376999999955e+07,3.262331126000000164e+07,3.674851661999999732e+07,3.729124652000000328e+07,3.387800424000000209e+07,3.734005210000000149e+07,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,7.950000000000000178e+00,6.520999999999999375e+01,3.291100000000000136e+02,1.036569999999999936e+03,3.911159999999999854e+03,9.725440000000000509e+03,2.691104999999999927e+04,6.339611000000000058e+04,1.578541099999999860e+05,3.132843800000000047e+05,5.658801899999999441e+05,9.233899000000000233e+05,1.426418979999999981e+06,2.178653770000000019e+06,3.011524149999999907e+06,4.487128049999999814e+06,5.973860030000000261e+06,8.157051629999999888e+06,1.037840766000000015e+07,1.259751235999999940e+07,1.634554805000000075e+07,2.046439596000000089e+07,2.369174351000000164e+07,2.731322016000000015e+07,3.415433385000000149e+07,3.625044986999999732e+07,4.318136402000000328e+07,4.592983468999999762e+07,4.924018289999999851e+07,5.362414284000000358e+07,5.965940392999999970e+07,6.246690867999999970e+07,6.578972045000000298e+07,0.000000000000000000e+00,8.000000000000000444e-01,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,3.149999999999999911e+00,6.435999999999999943e+01,2.936200000000000045e+02,1.098819999999999936e+03,1.591660000000000082e+03,5.554970000000000255e+03,1.928802999999999884e+04,4.531909999999999854e+04,8.304827000000000407e+04,1.691117000000000116e+05,3.206530499999999884e+05,5.693084699999999721e+05,8.576787700000000186e+05,1.319464520000000019e+06,1.981251770000000019e+06,2.984364700000000186e+06,3.910641029999999795e+06,5.614692370000000112e+06,7.236591459999999963e+06,9.918629019999999553e+06,1.274156167999999970e+07,1.660419006000000052e+07,2.075878071000000089e+07,2.681268376999999955e+07,3.042466432999999821e+07,3.698279152000000328e+07,4.876455547999999672e+07,5.315487014000000060e+07,5.880806688000000268e+07,6.763501482999999821e+07,7.752425043999999762e+07,9.181233959999999404e+07,1.033072790600000024e+08,1.111093009899999946e+08,1.208514124599999934e+08,0.000000000000000000e+00,0.000000000000000000e+00,4.650000000000000355e+00,2.678999999999999915e+01,8.251999999999999602e+01,2.822699999999999818e+02,7.420199999999999818e+02,1.858970000000000027e+03,5.308319999999999709e+03,1.080729000000000087e+04,2.866504000000000087e+04,5.363443000000000029e+04,1.111747400000000052e+05,2.005534400000000023e+05,3.212536300000000047e+05,5.348966500000000233e+05,8.208383399999999674e+05,1.273014149999999907e+06,1.690313709999999963e+06,2.750845709999999963e+06,3.545856709999999963e+06,5.240852990000000224e+06,6.643161219999999739e+06,8.713185310000000522e+06,1.219373667999999970e+07,1.541093030000000075e+07,1.979745698999999836e+07,2.582695701000000164e+07,3.211928469999999925e+07,4.061021978999999911e+07,5.340181142999999970e+07,5.915186307999999821e+07,7.401859295000000298e+07,8.763754459000000358e+07,1.029986682500000000e+08,1.197234840199999958e+08,1.454706896800000072e+08,1.603665652500000000e+08,1.948203179399999976e+08,2.291245150900000036e+08,2.399999999999999911e-01,1.469999999999999973e+00,1.242999999999999972e+01,4.496999999999999886e+01,1.942899999999999920e+02,5.660199999999999818e+02,1.179160000000000082e+03,3.429949999999999818e+03,7.619640000000000327e+03,1.969750999999999840e+04,3.595597000000000116e+04,7.521542999999999302e+04,1.391980299999999988e+05,2.464936199999999953e+05,3.561604400000000023e+05,6.215860999999999767e+05,9.302286500000000233e+05,1.377345000000000000e+06,2.163552350000000093e+06,2.993904330000000075e+06,3.942362089999999851e+06,5.665247820000000298e+06,7.465736049999999814e+06,1.005576190000000037e+07,1.305085057000000030e+07,1.636839048000000045e+07,2.195480505999999866e+07,2.634233803999999911e+07,3.503572760000000149e+07,4.407963167999999970e+07,5.368130060999999940e+07,6.359639960000000149e+07,8.072661456999999285e+07,9.615992128000000119e+07,1.225397104699999988e+08,1.419793820300000012e+08,1.641925042500000000e+08,1.951492562400000095e+08,2.242197358199999928e+08,2.628089998700000048e+08,0.000000000000000000e+00,1.510000000000000009e+00,1.483000000000000007e+01,3.878999999999999915e+01,2.058600000000000136e+02,5.290499999999999545e+02,1.641069999999999936e+03,3.479769999999999982e+03,8.451510000000000218e+03,2.076800999999999840e+04,4.018184999999999854e+04,8.208253999999999360e+04,1.404800799999999872e+05,2.436856099999999860e+05,4.124920499999999884e+05,6.649932199999999721e+05,9.531693800000000047e+05,1.411966820000000065e+06,2.159567180000000168e+06,3.007399950000000186e+06,4.081971089999999851e+06,5.626374969999999739e+06,7.855479589999999851e+06,1.003434949000000022e+07,1.319621780000000075e+07,1.703552298000000045e+07,2.091285466000000015e+07,2.605280910000000149e+07,3.336931567000000179e+07,4.052464832999999821e+07,4.837025157999999821e+07,5.997432497999999672e+07,7.277452998999999464e+07,7.941537862000000477e+07,9.026260346999999881e+07,1.064915725000000000e+08,1.204667195199999958e+08,1.188280843400000036e+08,1.350282706200000048e+08,1.548363070300000012e+08,0.000000000000000000e+00,8.499999999999999778e-01,1.586999999999999922e+01,5.335999999999999943e+01,2.036100000000000136e+02,5.238300000000000409e+02,1.485160000000000082e+03,3.692699999999999818e+03,8.968120000000000800e+03,1.838275000000000000e+04,4.323762000000000262e+04,8.619836999999999534e+04,1.672642399999999907e+05,2.801897399999999907e+05,4.471601599999999744e+05,7.383473699999999953e+05,1.101433739999999991e+06,1.766777280000000028e+06,2.435431919999999925e+06,3.549959109999999870e+06,4.762771690000000410e+06,6.288887519999999553e+06,8.239793219999999739e+06,1.164238443999999948e+07,1.402135603999999911e+07,1.686030023999999836e+07,2.043880450000000000e+07,2.465249780000000075e+07,3.078122842999999970e+07,3.552105392000000179e+07,3.927588261999999732e+07,4.548524156000000238e+07,4.621074300999999791e+07,4.635427199000000209e+07,4.622368572999999672e+07,4.000755597999999672e+07,3.802838689000000060e+07,3.584516010000000149e+07,2.738546457999999821e+07,2.507698653999999911e+07,0.000000000000000000e+00,2.000000000000000000e+00,1.810000000000000142e+01,7.976000000000000512e+01,1.953600000000000136e+02,6.234199999999999591e+02,1.541579999999999927e+03,3.839500000000000000e+03,9.748750000000000000e+03,2.014054000000000087e+04,4.627347000000000116e+04,9.869022000000000116e+04,1.809798099999999977e+05,3.136955700000000070e+05,5.258028399999999674e+05,8.681904899999999907e+05,1.266366679999999935e+06,1.968302860000000102e+06,3.015119870000000112e+06,3.771142310000000056e+06,5.050009629999999888e+06,6.199676570000000298e+06,7.984430820000000298e+06,1.009186752999999933e+07,1.183839388000000082e+07,1.516544206000000052e+07,1.720028828000000119e+07,1.703234476000000164e+07,2.114160305000000075e+07,2.026723710999999940e+07,2.157400987000000104e+07,2.230140591000000015e+07,1.672703972000000067e+07,1.674511580000000075e+07,1.320723494999999925e+07,1.140121178999999911e+07,7.146450089999999851e+06,6.458280019999999553e+06,2.987219600000000093e+06,4.316521030000000261e+06,3.499999999999999778e-01,3.459999999999999964e+00,2.287000000000000099e+01,7.089000000000000057e+01,2.254499999999999886e+02,5.683600000000000136e+02,1.685200000000000045e+03,4.173760000000000218e+03,1.059545999999999913e+04,2.476911000000000058e+04,5.028851000000000204e+04,1.061349799999999959e+05,2.147785599999999977e+05,3.629904899999999907e+05,5.668610699999999488e+05,9.222156500000000233e+05,1.339578800000000047e+06,1.852380270000000019e+06,2.462299060000000056e+06,3.363033939999999944e+06,4.233027059999999590e+06,5.620846169999999925e+06,6.959325790000000037e+06,7.660129450000000186e+06,8.807970130000000820e+06,1.001593297000000067e+07,1.150732183000000007e+07,1.234135905000000075e+07,1.136642992999999970e+07,1.172228556000000052e+07,1.056485149000000022e+07,7.914543110000000335e+06,5.362557009999999776e+06,3.641798890000000130e+06,5.303451280000000261e+06,2.078207340000000084e+06,1.920515709999999963e+06,3.710146099999999860e+05,1.340146570000000065e+06,0.000000000000000000e+00,0.000000000000000000e+00,3.069999999999999840e+00,1.726999999999999957e+01,8.767000000000000171e+01,2.220900000000000034e+02,6.573300000000000409e+02,1.601519999999999982e+03,4.056659999999999854e+03,1.223879000000000087e+04,2.721277999999999884e+04,5.697408000000000175e+04,1.199619299999999930e+05,2.191119400000000023e+05,3.648681900000000023e+05,5.924640899999999674e+05,9.044826800000000512e+05,1.245929729999999981e+06,1.869267719999999972e+06,2.413311160000000149e+06,2.919273020000000019e+06,3.859568350000000093e+06,4.883969940000000410e+06,5.943227549999999814e+06,6.400594240000000224e+06,6.700296099999999627e+06,7.550249639999999665e+06,7.447632870000000112e+06,7.429817290000000037e+06,6.081131570000000298e+06,5.630375709999999963e+06,4.806752580000000075e+06,4.989217230000000447e+06,2.966468339999999851e+06,1.118517179999999935e+06,6.579976700000000419e+05,9.728583800000000047e+05,1.935053099999999977e+05,0.000000000000000000e+00,3.247443499999999767e+05,2.525277000000000044e+04,2.500000000000000000e-01,3.569999999999999840e+00,1.528999999999999915e+01,6.984999999999999432e+01,2.648999999999999773e+02,6.973500000000000227e+02,1.652440000000000055e+03,4.768029999999999745e+03,1.298873999999999978e+04,3.191815999999999985e+04,6.634707000000000698e+04,1.179406300000000047e+05,2.214101400000000140e+05,3.617304500000000116e+05,5.146840200000000186e+05,8.018732700000000186e+05,1.232090729999999981e+06,1.629009199999999953e+06,2.268047879999999888e+06,2.668326319999999832e+06,3.460764600000000093e+06,4.097694390000000130e+06,4.532456190000000410e+06,5.178768959999999963e+06,4.882558790000000037e+06,5.575227509999999776e+06,5.026675910000000149e+06,5.220841919999999925e+06,4.220461030000000261e+06,4.281989280000000261e+06,2.920979899999999907e+06,2.312958500000000000e+06,2.624305689999999944e+06,8.780821700000000419e+05,5.232488599999999860e+05,6.296296600000000326e+05,0.000000000000000000e+00,1.267870000000000073e+04,1.471467900000000081e+05,0.000000000000000000e+00,0.000000000000000000e+00,4.620000000000000107e+00,1.626000000000000156e+01,7.542000000000000171e+01,1.825099999999999909e+02,6.087300000000000182e+02,2.043160000000000082e+03,6.080319999999999709e+03,1.454889999999999964e+04,3.210574000000000160e+04,6.468345999999999913e+04,1.265439499999999971e+05,2.057767099999999919e+05,3.407145200000000186e+05,5.488917900000000373e+05,7.812068599999999860e+05,1.137165689999999944e+06,1.596975830000000075e+06,1.872314479999999981e+06,2.502849189999999944e+06,2.842591370000000112e+06,3.469055620000000112e+06,3.890208950000000186e+06,3.884307720000000205e+06,4.220786200000000186e+06,4.082780169999999925e+06,4.153340819999999832e+06,3.960615680000000168e+06,3.458853509999999776e+06,2.495069209999999963e+06,1.489639439999999944e+06,1.383877560000000056e+06,3.837368900000000140e+05,3.284597700000000186e+05,5.554457299999999814e+05,7.750472999999999593e+04,0.000000000000000000e+00,0.000000000000000000e+00,6.839988999999999942e+04,0.000000000000000000e+00,1.900000000000000022e-01,2.180000000000000160e+00,1.698000000000000043e+01,6.825000000000000000e+01,2.844300000000000068e+02,6.408200000000000500e+02,2.094329999999999927e+03,7.105439999999999600e+03,1.476609000000000015e+04,3.220631999999999971e+04,6.311776000000000204e+04,1.120245000000000000e+05,1.887483299999999872e+05,3.143596900000000023e+05,5.243749499999999534e+05,7.717837500000000000e+05,1.016341300000000047e+06,1.302336290000000037e+06,1.651449830000000075e+06,2.070487100000000093e+06,2.451744689999999944e+06,2.598520419999999925e+06,3.034233509999999776e+06,3.673489890000000130e+06,3.132934370000000112e+06,2.709940390000000130e+06,3.287013589999999851e+06,2.337496290000000037e+06,1.654650770000000019e+06,1.377569139999999898e+06,5.271488699999999953e+05,3.604836799999999930e+05,4.528409699999999721e+05,5.346470999999999913e+04,7.759453999999999360e+04,2.529197999999999956e+04,3.714372000000000116e+04,1.260913999999999942e+05,0.000000000000000000e+00,7.488194000000000233e+04,8.399999999999999689e-01,2.839999999999999858e+00,2.048000000000000043e+01,8.045999999999999375e+01,2.465500000000000114e+02,6.644600000000000364e+02,2.525070000000000164e+03,6.403960000000000036e+03,1.460788999999999942e+04,3.252197000000000116e+04,6.355812000000000262e+04,1.119109799999999959e+05,1.879273399999999965e+05,3.042362199999999721e+05,4.643220900000000256e+05,6.972535600000000559e+05,9.591860799999999581e+05,1.265317500000000000e+06,1.412026610000000102e+06,1.797596409999999916e+06,2.141001009999999776e+06,2.251922049999999814e+06,2.845302649999999907e+06,2.653161189999999944e+06,2.975103839999999851e+06,2.609156049999999814e+06,1.765786209999999963e+06,1.761795729999999981e+06,1.049051090000000084e+06,8.014782199999999721e+05,7.722117399999999907e+05,3.573930200000000186e+05,1.314656999999999971e+04,9.664419999999999709e+04,0.000000000000000000e+00,1.167704999999999927e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,4.500000000000000111e-01,1.310000000000000053e+00,1.921000000000000085e+01,5.936999999999999744e+01,2.165999999999999943e+02,9.568400000000000318e+02,2.500630000000000109e+03,6.487310000000000400e+03,1.457343000000000029e+04,3.175409000000000015e+04,6.189383999999999651e+04,1.009122200000000012e+05,2.187877300000000105e+05,3.075781099999999860e+05,4.656411500000000233e+05,6.262011600000000326e+05,8.573248000000000466e+05,1.255623459999999963e+06,1.458025010000000009e+06,1.618932719999999972e+06,1.814487429999999935e+06,2.151687669999999925e+06,2.330875899999999907e+06,2.138571529999999795e+06,1.985970810000000056e+06,1.713073469999999972e+06,1.242037489999999991e+06,1.395006000000000000e+06,6.782493499999999767e+05,3.359189600000000210e+05,7.885130999999999767e+04,4.639205999999999767e+04,3.744176999999999680e+04,1.857238999999999942e+04,0.000000000000000000e+00,2.080106999999999971e+04,3.328254000000000087e+04,3.339326000000000204e+04,0.000000000000000000e+00,0.000000000000000000e+00,2.200000000000000011e-01,3.009999999999999787e+00,2.228000000000000114e+01,6.445999999999999375e+01,2.638199999999999932e+02,8.504700000000000273e+02,2.368590000000000146e+03,5.727329999999999927e+03,1.421594000000000051e+04,2.765320000000000073e+04,6.025570999999999913e+04,1.017712700000000041e+05,1.609074599999999919e+05,3.002064199999999837e+05,4.481973599999999860e+05,5.622258100000000559e+05,8.140413900000000140e+05,1.031620290000000037e+06,1.260118010000000009e+06,1.428452770000000019e+06,1.502275669999999925e+06,1.585969449999999953e+06,1.579364209999999963e+06,1.372639139999999898e+06,1.467888800000000047e+06,1.353504209999999963e+06,1.114469679999999935e+06,3.279019899999999907e+05,4.125178599999999860e+05,2.615868500000000058e+05,1.209636000000000058e+04,0.000000000000000000e+00,5.631998000000000320e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,4.099999999999999756e-01,4.669999999999999929e+00,1.925000000000000000e+01,7.439000000000000057e+01,3.459100000000000250e+02,8.296200000000000045e+02,2.491230000000000018e+03,6.533069999999999709e+03,1.341136000000000058e+04,3.171245000000000073e+04,6.859847000000000116e+04,1.061876699999999983e+05,1.700640799999999872e+05,2.872091699999999837e+05,4.277786500000000233e+05,5.685768100000000559e+05,8.091373399999999674e+05,8.896076700000000419e+05,1.122313669999999925e+06,1.277087760000000009e+06,1.395551800000000047e+06,1.389456459999999963e+06,1.333730459999999963e+06,1.037774359999999986e+06,1.148834750000000000e+06,9.230418599999999860e+05,5.218519400000000023e+05,4.134112899999999790e+05,2.056732999999999884e+05,3.100630000000000109e+03,2.414263000000000102e+04,4.375862999999999738e+04,7.695750000000000000e+03,0.000000000000000000e+00,3.963510000000000218e+03,0.000000000000000000e+00,4.042276000000000204e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,2.600000000000000089e-01,3.709999999999999964e+00,1.593999999999999950e+01,9.340000000000000568e+01,3.010199999999999818e+02,9.655900000000000318e+02,2.366610000000000127e+03,6.542760000000000218e+03,1.442786000000000058e+04,3.048840000000000146e+04,5.994369999999999709e+04,1.121193800000000047e+05,1.720919299999999930e+05,2.610625299999999988e+05,3.996317500000000000e+05,4.836627000000000116e+05,6.692221099999999860e+05,9.855974399999999441e+05,9.744165699999999488e+05,9.795820400000000373e+05,1.376267639999999898e+06,1.192673429999999935e+06,1.231615659999999916e+06,1.210392449999999953e+06,7.007354300000000512e+05,6.176433499999999767e+05,5.022738400000000256e+05,1.684441600000000035e+05,1.627452799999999988e+05,5.735079999999999927e+03,3.022609000000000015e+04,0.000000000000000000e+00,4.046986000000000058e+04,0.000000000000000000e+00,7.691930000000000291e+03,0.000000000000000000e+00,2.544070000000000164e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,7.399999999999999911e-01,3.700000000000000178e+00,1.448000000000000043e+01,9.614000000000000057e+01,3.486600000000000250e+02,1.041589999999999918e+03,2.734480000000000018e+03,6.849789999999999964e+03,1.495532999999999993e+04,2.665527999999999884e+04,5.374511000000000058e+04,9.716291999999999825e+04,1.477677399999999907e+05,2.397999800000000105e+05,3.228697700000000186e+05,4.857457899999999790e+05,6.985521500000000233e+05,8.609797399999999907e+05,8.899009899999999907e+05,9.789255899999999674e+05,1.137230340000000084e+06,1.078933810000000056e+06,8.031987900000000373e+05,5.888730200000000186e+05,6.437589200000000419e+05,2.304751799999999930e+05,2.666191300000000047e+05,3.254466599999999744e+05,3.637312000000000262e+04,2.602281000000000131e+04,3.372236000000000058e+04,6.364329999999999927e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.040000000000000036e+00,4.629999999999999893e+00,1.826999999999999957e+01,1.071500000000000057e+02,3.290199999999999818e+02,1.215470000000000027e+03,3.046199999999999818e+03,7.074640000000000327e+03,1.360765999999999985e+04,2.902095000000000073e+04,5.454762000000000262e+04,9.356749000000000524e+04,1.669211499999999942e+05,2.209087399999999907e+05,3.891423200000000070e+05,4.647152500000000000e+05,6.139516099999999860e+05,5.835171300000000047e+05,8.081372099999999627e+05,9.503016099999999860e+05,9.389136600000000326e+05,7.301122099999999627e+05,6.666581500000000233e+05,4.205978099999999977e+05,2.285527099999999919e+05,2.525168900000000140e+05,9.604269999999999709e+04,3.017445999999999913e+04,1.635176000000000058e+05,7.331924000000000524e+04,6.382680000000000291e+03,0.000000000000000000e+00,3.457595999999999913e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,2.970000000000000195e+00,2.125000000000000000e+01,1.049899999999999949e+02,3.062099999999999795e+02,1.166559999999999945e+03,2.746920000000000073e+03,6.302300000000000182e+03,1.457055999999999949e+04,3.007897000000000116e+04,5.295302999999999884e+04,9.087238999999999942e+04,1.500635700000000070e+05,2.210336499999999942e+05,3.165652600000000093e+05,4.472880599999999977e+05,5.588181600000000326e+05,6.624505899999999674e+05,6.954991099999999860e+05,7.748904699999999721e+05,7.266893199999999488e+05,6.970649699999999721e+05,5.157070599999999977e+05,3.643127600000000093e+05,2.550309299999999930e+05,1.781591799999999930e+05,3.639381999999999971e+04,9.180605000000000291e+04,6.527333000000000175e+04,1.916718999999999869e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,4.600000000000000200e-01,1.870000000000000107e+00,1.828000000000000114e+01,9.098999999999999488e+01,3.556399999999999864e+02,1.139960000000000036e+03,2.751769999999999982e+03,8.156859999999999673e+03,1.397087000000000080e+04,3.269352999999999884e+04,5.567736000000000058e+04,9.069139999999999418e+04,1.456655899999999965e+05,2.072583999999999942e+05,2.971654500000000116e+05,4.139323699999999953e+05,5.633821999999999534e+05,7.479395000000000000e+05,5.402685200000000186e+05,6.706494399999999441e+05,5.918190300000000279e+05,5.770566899999999441e+05,4.851276300000000047e+05,4.115783400000000256e+05,2.199704800000000105e+05,1.459741400000000140e+05,9.781211999999999534e+04,4.935354000000000087e+04,1.025309400000000023e+05,6.205399999999999636e+03,2.532909999999999854e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,7.900000000000000355e-01,3.370000000000000107e+00,2.025000000000000000e+01,1.437199999999999989e+02,3.724300000000000068e+02,1.436420000000000073e+03,3.225989999999999782e+03,7.110869999999999891e+03,1.361760000000000036e+04,2.733879999999999927e+04,5.493955999999999767e+04,8.202688000000000466e+04,1.329114700000000012e+05,2.135662300000000105e+05,2.971522600000000093e+05,3.839646400000000140e+05,4.871372399999999907e+05,5.157707000000000116e+05,6.421587700000000186e+05,5.612103900000000140e+05,5.964959100000000326e+05,4.640062800000000279e+05,5.063308300000000163e+05,2.927745399999999790e+05,1.401602600000000093e+05,7.189580999999999767e+04,8.537458999999999651e+04,5.703544000000000233e+04,1.095003500000000058e+05,0.000000000000000000e+00,0.000000000000000000e+00,2.019049000000000160e+04,0.000000000000000000e+00,5.594279999999999745e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.290000000000000036e+00,4.150000000000000355e+00,3.114000000000000057e+01,1.290500000000000114e+02,4.174800000000000182e+02,1.396869999999999891e+03,2.934030000000000200e+03,7.594060000000000400e+03,1.295977000000000044e+04,3.179322000000000116e+04,5.335301999999999680e+04,9.069994000000000233e+04,1.344907500000000000e+05,1.847848099999999977e+05,2.860257299999999814e+05,3.268412800000000279e+05,4.008794699999999721e+05,4.993385200000000186e+05,4.803370700000000070e+05,4.752754199999999837e+05,4.473338599999999860e+05,4.332114099999999744e+05,2.947612500000000000e+05,2.515476300000000047e+05,1.207053000000000029e+05,6.241079999999999927e+03,1.411515700000000070e+05,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,4.302029999999999745e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.149999999999999911e+00,5.110000000000000320e+00,2.926999999999999957e+01,1.641599999999999966e+02,5.094100000000000250e+02,1.203509999999999991e+03,2.851480000000000018e+03,7.483210000000000036e+03,1.535001000000000022e+04,2.959084999999999854e+04,5.124763999999999942e+04,7.365175000000000000e+04,1.215901100000000006e+05,1.877909500000000116e+05,2.672045800000000163e+05,3.275305599999999977e+05,3.842562100000000210e+05,3.795080800000000163e+05,4.202692999999999884e+05,4.731754000000000233e+05,4.148394299999999930e+05,3.116916599999999744e+05,2.956074000000000233e+05,1.689819599999999919e+05,5.998620999999999913e+04,2.043027000000000044e+04,2.484509999999999854e+04,2.913440000000000146e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,3.128610000000000127e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.200000000000000178e-01,7.190000000000000391e+00,4.584000000000000341e+01,1.464499999999999886e+02,4.711600000000000250e+02,1.295859999999999900e+03,3.389340000000000146e+03,7.030890000000000327e+03,1.291251000000000022e+04,3.042868999999999869e+04,4.862819999999999709e+04,8.201338000000000466e+04,1.208855299999999988e+05,1.865530199999999895e+05,2.340148599999999860e+05,2.688850599999999977e+05,4.447762999999999884e+05,4.266998800000000047e+05,4.868926699999999837e+05,3.602827999999999884e+05,2.664481599999999744e+05,1.919720499999999884e+05,1.155105200000000041e+05,1.461515299999999988e+05,3.099845999999999913e+04,4.124901999999999680e+04,0.000000000000000000e+00,0.000000000000000000e+00,6.175859999999999673e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,2.100000000000000089e+00,5.879999999999999893e+00,4.788000000000000256e+01,1.891500000000000057e+02,4.458700000000000045e+02,1.571960000000000036e+03,2.943739999999999782e+03,7.521850000000000364e+03,1.446223999999999978e+04,2.929066999999999825e+04,4.711773000000000320e+04,8.434891999999999825e+04,1.069348099999999977e+05,1.607360700000000070e+05,1.930961600000000035e+05,2.986682000000000116e+05,2.764019500000000116e+05,3.506403400000000256e+05,2.671982800000000279e+05,2.520187999999999884e+05,1.667866300000000047e+05,9.395188999999999942e+04,1.142200700000000070e+05,6.235602999999999884e+04,3.558894999999999709e+04,2.554625999999999840e+04,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.580000000000000071e+00,9.449999999999999289e+00,4.995000000000000284e+01,1.921500000000000057e+02,5.379700000000000273e+02,1.353349999999999909e+03,3.137789999999999964e+03,7.080750000000000000e+03,1.215398999999999978e+04,2.591570000000000073e+04,4.565525000000000000e+04,6.381312000000000262e+04,9.985183000000000175e+04,1.441658900000000140e+05,1.845610899999999965e+05,1.921083500000000058e+05,1.891582300000000105e+05,1.734965700000000070e+05,2.452101000000000058e+05,1.381097600000000093e+05,7.010361999999999534e+04,4.209290000000000146e+04,3.574106999999999971e+04,3.839852999999999884e+04,6.356900000000000546e+02,4.148680000000000291e+03,5.975760000000000218e+03,9.967879999999999200e+03,6.744310000000000400e+03,3.587630000000000109e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.300000000000000044e+00,6.299999999999999822e+00,4.967000000000000171e+01,1.688499999999999943e+02,6.383799999999999955e+02,1.408309999999999945e+03,2.829539999999999964e+03,6.842140000000000327e+03,1.382410000000000036e+04,2.448095000000000073e+04,3.883069999999999709e+04,6.314708000000000175e+04,8.708683999999999651e+04,1.472242399999999907e+05,1.365275599999999977e+05,1.425113399999999965e+05,1.631598099999999977e+05,1.178861499999999942e+05,9.764238999999999942e+04,9.363442999999999302e+04,7.992647000000000116e+04,1.094184000000000015e+04,1.409800000000000000e+04,9.709299999999999272e+03,0.000000000000000000e+00,0.000000000000000000e+00,1.504549999999999955e+03,5.037099999999999795e+02,3.342869999999999891e+03,9.965599999999999454e+02,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,5.999999999999999778e-01,3.580000000000000071e+00,3.324000000000000199e+01,1.425200000000000102e+02,3.500799999999999841e+02,9.498099999999999454e+02,2.337400000000000091e+03,4.893000000000000000e+03,1.097992000000000007e+04,1.810943000000000029e+04,3.102300999999999840e+04,4.799855000000000291e+04,6.338926999999999680e+04,1.022162299999999959e+05,8.850219999999999709e+04,1.188817899999999936e+05,1.212333699999999953e+05,9.129672999999999593e+04,7.934397999999999593e+04,5.006680999999999767e+04,4.178816000000000349e+04,1.952693999999999869e+04,5.400310000000000400e+03,3.648199999999999932e+02,4.664050000000000182e+03,0.000000000000000000e+00,3.800239999999999782e+03,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,1.108009999999999991e+03,1.771999999999999886e+02,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00,0.000000000000000000e+00"
earea = earea.split(",")

In [5]:
earea = [float(i) for i in earea]
earea = np.asfarray(earea)

In [6]:
#generates ns^ for a single pulsar a single season and a single energy

@njit(fastmath=True)
def dfde(e_nu, gamma=1, phi0 = 1e-40):
    '''
    Parameters
    ----------
    e_nu : float
        The neutrino energy in eV
    
    gamma : float
        The spectral index of the neutrino flux. The default is 1.
    
    phi0 : float (optional)
        The normalization constant of the neutrino flux in eV^-1. The default is 1e-40.
    
    Returns
    -------
    float
        The differential flux of neutrinos
    '''

    return phi0 * ((e_nu/(10**14))**gamma)      #eV^-1



@njit(fastmath=True)
def ns_singleseason_sing_psr_HAT(dec,enu, gamma, phi0 = 1, season=0):
    '''
    This function returns the number of signal events for a single pulsar as in EQN3 of 2205.15963
    
    
    FOR SEASON 0 ONLY
    -------------------

    Parameters
    ----------
    dec : float
        The declination of the pulsar in radians
    
    enu : float
        The neutrino energy in eV

    gamma : float
        The spectral index of the neutrino flux

    phi0 : float (optional)
        The normalization constant of the neutrino flux
    
    Returns
    -------
    float
        The number of signal events for the given parameters
    '''


    tt_upt = t_upt
    if enu <= 1e11 or enu >= 1e19:
        return 0.0
    else:
        k=0
        l=0
        for i in range(0, len(e_nu_wall)):
            if e_nu_wall[i] <= enu and e_nu_wall[i+1] > enu:
                for j in range(0, len(dec_nu)):
                    if dec_nu[j] <= dec and dec_nu[j+1] > dec:
                        k=i
                        l=j
                        break
                break

        temp_ea = np.asarray(earea)[l*40 + k]
        return tt_upt * temp_ea * dfde(enu, gamma, phi0)     #in s cm2 eV


In [7]:
len(dec_nu)

51

In [8]:
len(e_nu_wall)

41

In [45]:
phi0t = 1e-30
season = 0


dec = 62.26928
print("Decl = ", dec)
tmp = [ns_singleseason_sing_psr_HAT(dec, all_enu[i], -1, phi0t ,season=season) for i in range(len(all_enu))]

#UNEQUAL BINWIDTHS IN LINEAR SPACE BUT EQUAL IN LOGARITHMIC SPACE
print("ns^ = ",np.trapz(y=tmp, dx=np.diff(all_enu)))

#EQUAL BINWIDTHS IN BOTH LINEAR 10^9.2 AND LOGARITHMIC SPACE of 0.2
print("ns^ = ",np.trapz(y=tmp, dx = 1e9* (10**0.2)))

Decl =  62.26928
ns^ =  0.0029378291831797018
ns^ =  2.1708436104051152e-07
